In [1]:
import torch
import os
from torch import nn
import torch.nn.functional as F 
import pickle
from torchsummary import summary
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import MultiStepLR
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from tqdm import tqdm
from model.res3net import Res3Net, BasicBlock
from model.res3netpre import Res3NetPre, BasicBlockPre

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def make_data_label(lst_of_dic, do_label=True):
    data = torch.empty((0, 3, 32, 32))
    if do_label:
        label = torch.empty(0, dtype=torch.long)
    for dic in lst_of_dic:
        # Convert the data to a tensor and reshape
        cur_data = torch.tensor(dic[b'data']).reshape(10000, 3, 32, 32)
        data = torch.cat((data, cur_data), dim=0)
        if do_label:
            cur_label = torch.tensor(dic[b'labels'], dtype=torch.long).reshape(10000)
            label = torch.cat((label, cur_label), dim=0)
    # print(data[1])
    if do_label:
        return data, label
    else:
        return data

class cifar_dataset(Dataset):
    def __init__(self, data, label, transform=None):
        """
        Args:
            data (Tensor): The raw image data with shape (n, 3, 32, 32) and pixel values in [0, 255].
            label (Tensor): The corresponding labels.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # Convert data to float and normalize pixel values to [0, 1]
        self.transform = transforms.Compose([transforms.Normalize(mean = [0.4914, 0.4822, 0.4465], std = [0.2470, 0.2435, 0.2616])])
        self.data = data / 255
        self.label = label

    def __getitem__(self, index):
        img, lbl = self.data[index], self.label[index]
        return img, lbl

    def __len__(self):
        return self.data.shape[0]

In [3]:
root = r'deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_'
lst = []
for i in range(5):
    lst.append(unpickle(root+str(i+1)))
#lst

In [4]:
data, label = make_data_label(lst)
dataset = cifar_dataset(data, label)
split = int(len(dataset)*0.6)
indices = list(range(len(dataset)))
train_set  = Subset(dataset, indices[:split])
valid_set = Subset(dataset, indices[split:])
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [5]:
class BasicBlock2(nn.Module):
    expansion = 1


    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock2, self).__init__()

        DROPOUT = 0.1

        # self.bn0 = nn.BatchNorm2d(in_planes)


        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(DROPOUT)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(DROPOUT)


        self.bn3 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(DROPOUT)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes),
                nn.Dropout(DROPOUT)
            )

    def forward(self, x):
        # x = F.relu(self.bn0(x))
        out = F.relu(self.dropout(self.bn1(self.conv1(x))))
        out = F.relu(self.dropout(self.bn2(self.conv2(out))))
        out = out + self.shortcut(x)
        # out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks=[2, 2, 2, 2], num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        final = 381
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, final, num_blocks[2], stride=2)
        # self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # self.linear1 = nn.Linear(final*block.expansion, num_classes)
        self.linear1 = nn.Linear(final*block.expansion, 128)
        self.linear2 = nn.Linear(128, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        # out = self.layer4(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        # print(out.shape)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        return F.log_softmax(out, dim=-1)
        # return out


    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Importing Model and printing Summary
model = Res3Net(BasicBlock).to(device)
summary(model, input_size=(3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
           Dropout-5           [-1, 64, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,864
      BatchNorm2d-10           [-1, 64, 32, 32]             128
          Dropout-11           [-1, 64, 32, 32]               0
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,

In [6]:
torch.cuda.empty_cache()

In [7]:
cwd = os.getcwd()
def model_testing(model, device, test_dataloader, test_acc, test_losses, misclassified = []):

    model.eval()
    test_loss = 0
    correct = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    # label = 0
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    with torch.no_grad():

        for index, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)

            for d,i,j in zip(data, pred, target):
                if i != j:
                    misclassified.append([d.cpu(),i[0].cpu(),j.cpu()])

            test_loss += F.nll_loss(output, target, reduction='sum').item()
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_dataloader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

    test_acc.append(100. * correct / len(test_dataloader.dataset))
    return misclassified

def model_training(model, device, train_dataloader, optimizer, train_acc, train_losses):

    model.train()
    # pbar = tqdm(train_dataloader)
    correct = 0
    processed = 0
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)


        train_losses.append(loss)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        # print statistics
        running_loss += loss.item()
        # pbar.set_description(desc=f'Loss={loss.item():.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)
    print(f'Loss={loss.item():.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
# scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

train_acc = []
train_losses = []
valid_acc = []
valid_losses = []

EPOCHS = 40

for i in range(EPOCHS):

    print(f'EPOCHS : {i}')
    model_training(model, device, train_loader, optimizer, train_acc, train_losses)
    scheduler.step(train_losses[-1])
    misclassified = model_testing(model, device, valid_loader, valid_acc, valid_losses)

C:\Users\seer2\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


EPOCHS : 0
Loss=1.6463 Batch_id=468 Accuracy=28.00

Test set: Average loss: 2.1231, Accuracy: 4835/20000 (24.18%)

EPOCHS : 1
Loss=1.4706 Batch_id=468 Accuracy=44.97

Test set: Average loss: 1.5490, Accuracy: 9258/20000 (46.29%)

EPOCHS : 2
Loss=1.1252 Batch_id=468 Accuracy=55.66

Test set: Average loss: 1.3580, Accuracy: 10363/20000 (51.81%)

EPOCHS : 3


In [71]:
data_no_label = unpickle(r'deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl')
ids = data_no_label[b'ids']
data_no_label = make_data_label([data_no_label], do_label=False)
dummy_label = torch.ones((data_no_label.shape[0], 1))*-1
test = cifar_dataset(data_no_label, dummy_label)
test_loader = DataLoader(test, batch_size=1, shuffle=False)
valid_loader2 = DataLoader(valid_set, batch_size=1, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
result = []
for img, _ in test_loader:
    model.eval()
    img = img.to(device)
    y_pred = model(img)
    pred = int(y_pred.argmax(dim=1, keepdim=True))
    result.append(pred)


In [74]:
test[0]

(tensor([[[-1.6878, -1.9736, -1.9736,  ..., -1.9736, -1.7037, -1.9895],
          [-1.9895, -1.7037, -1.9895,  ..., -0.3224, -0.3542, -0.1795],
          [-0.3065, -0.3383, -0.6082,  ..., -0.2907, -0.4177, -0.4494],
          ...,
          [-1.9736, -1.9895, -1.7672,  ..., -1.7990, -1.9577, -1.9736],
          [-1.6561, -1.9577, -1.9736,  ...,  0.3762,  0.6461,  0.6937],
          [ 0.3127, -0.1319, -0.1954,  ..., -1.8942, -1.8942, -1.7672]],
 
         [[-1.9642, -1.9803, -1.7387,  ..., -1.7870, -1.9481, -1.9642],
          [-1.6743, -1.9642, -1.9803,  ...,  0.2422,  0.4677,  0.7254],
          [ 0.5965,  0.6931,  0.7898,  ..., -1.6743, -1.6904, -1.7387],
          ...,
          [ 0.9025,  0.3227,  0.7092,  ..., -0.3537, -0.7885, -0.9979],
          [-1.0623, -1.2717, -0.4020,  ..., -1.6260, -1.8192, -1.8998],
          [-1.5455, -1.6743, -1.8837,  ...,  0.9508,  0.7092,  1.0152]],
 
         [[ 1.0665,  0.8116,  1.1114,  ...,  0.1820, -0.2527, -0.4776],
          [-0.5525, -0.8074,

In [66]:
model_testing(model, device, valid_loader, valid_acc, valid_losses)


Test set: Average loss: 0.5249, Accuracy: 16965/20000 (84.83%)



[[tensor([[[ 0.8366,  0.8366,  0.8366,  ...,  0.7096,  0.6778,  0.6302],
           [ 0.8683,  0.8525,  0.8683,  ...,  0.7572,  0.7255,  0.6778],
           [ 0.8207,  0.8207,  0.8366,  ...,  0.7255,  0.6937,  0.6620],
           ...,
           [-0.2113, -0.1954, -0.1795,  ..., -0.4018, -0.4335, -0.3859],
           [-0.2113, -0.2113, -0.1954,  ..., -0.3700, -0.3700, -0.3700],
           [-0.3542, -0.4018, -0.3542,  ..., -0.5288, -0.5129, -0.5447]],
  
          [[ 0.8542,  0.8542,  0.8542,  ...,  0.7254,  0.6931,  0.6448],
           [ 0.8864,  0.8703,  0.8864,  ...,  0.7737,  0.7415,  0.6931],
           [ 0.8381,  0.8381,  0.8542,  ...,  0.7415,  0.7092,  0.6770],
           ...,
           [-0.2571, -0.2248, -0.2087,  ..., -0.4181, -0.4664, -0.4503],
           [-0.3376, -0.3215, -0.3215,  ..., -0.4503, -0.4825, -0.4986],
           [-0.4342, -0.4825, -0.4503,  ..., -0.6275, -0.5953, -0.6275]],
  
          [[ 1.1264,  1.1264,  1.1264,  ...,  0.9915,  0.9466,  0.9016],
           

In [72]:
def make_output(result):
    out = ['ID,Labels']
    for i in range(len(result)):
        cur = str(i) + str(',') + str(int(result[i]))
        out.append(cur)
    return '\n'.join(out)
with open("output.csv", "w", encoding="utf-8") as f:
    f.write(make_output(result))